In [229]:
import torch
from torch import nn
from torch import tensor
from torch import optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [8]:
samples = pd.read_csv('data/aqi_upsample.csv',index_col= 0)

In [184]:
torch.random.manual_seed(0)
c = torch.nn.LSTMCell(24,1)
hx= torch.randn(1,1)
cx= torch.randn(1,1)
inputs = torch.randn(1,72,24)
output = torch.tensor([])
with torch.no_grad():
    for i in range(len(inputs)):
        for j in range(inputs.shape[1]):
            x = torch.cat((hx,inputs[i,j,1:].unsqueeze(0)),1)
            hx, cx = c(x,(hx,cx))
            output = torch.cat((output,hx),1)

In [185]:
output

tensor([[-2.9377e-02, -3.2229e-02, -4.2702e-02, -7.8843e-01,  9.6925e-02,
          7.9377e-03, -1.0720e-01, -4.3579e-01,  7.4455e-02, -5.7638e-01,
          2.5740e-01, -1.2364e-03, -6.2777e-04, -9.3059e-04, -8.6659e-03,
          2.8637e-01, -2.0458e-01, -5.5831e-02, -6.2516e-01, -6.8382e-04,
         -6.6573e-01, -4.6355e-02, -3.9912e-05, -1.7753e-02, -3.0035e-01,
         -3.6432e-02, -4.4465e-01, -3.0688e-03,  3.4832e-02,  1.2146e-02,
          2.2520e-01, -1.5194e-04, -1.7131e-01, -7.1980e-02, -5.6197e-02,
         -9.7719e-03, -8.0705e-02, -2.0171e-01, -2.0209e-01, -1.0370e-01,
         -2.9572e-03, -2.1654e-01, -4.0712e-01, -2.7188e-02, -4.2719e-02,
         -1.6726e-01,  1.2039e-02, -2.3589e-04, -1.7912e-03, -2.9666e-02,
         -1.1500e-02,  6.9503e-01,  8.3289e-01, -4.0500e-01, -7.0315e-02,
          3.1722e-02,  7.0222e-01,  1.3136e-01,  4.8535e-02, -5.1856e-01,
         -3.5722e-02,  6.1767e-01, -2.8095e-03, -1.3378e-03, -1.1230e-01,
         -1.8689e-01, -3.2741e-02, -6.

### 只输入第一个aqi，其他置0

In [241]:
class NetWork(nn.Module):
    def __init__(self):
        super(NetWork,self).__init__()
        self.lstm = nn.LSTM(24,10,1)
        self.linear = nn.Linear(10,1)
        
    def forward(self, x):
        x,(h_1,c_1) = self.lstm(x)
        x = self.linear(x)
        return x

In [242]:
net = NetWork()
x = torch.from_numpy(samples.head(72).values).float()
net(x.unsqueeze(0))

tensor([[[-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],
         [-0.1928],


### 每个输出拿出来重新进模型

In [225]:
class NetWork(nn.Module):
    def __init__(self):
        super(NetWork,self).__init__()
        self.lstm = nn.LSTM(24,10,1)
        self.linear = nn.Linear(10,1)
        
    def forward(self, x, aqi):
        output = torch.tensor([aqi])
        for i in range(72):
            x_MeteFactor = torch.tensor(x[i, 1:])
            x_MeteFactor = x_MeteFactor.float()
            x_Onetime = torch.cat((output[-1].unsqueeze(0),x_MeteFactor)).unsqueeze(0)
            o, (_1, _2) = self.lstm(x_Onetime)
            o = self.linear(o)
            output = torch.cat((output,o[0].data),0)

In [213]:
rnn = nn.LSTM(24,10,1)
lnn = nn.Linear(10,1)

output = torch.tensor([samples.iloc[0,0]])
for i in range(72):
    x_MeteFactor = torch.tensor(samples.iloc[i,1:].values)
    x_MeteFactor = x_MeteFactor.float()
    x_Onetime = torch.cat((output[-1].unsqueeze(0),x_MeteFactor)).unsqueeze(0).unsqueeze(0)
    o, (_1, _2) = rnn(x_Onetime)
    o = lnn(o)
    output = torch.cat((output,o[0,0].data),0)

In [28]:
samples.head()

,aqi,tmp,v850,t925,p,t750,v750,v500,u850,u925,u750,u,t500,v,v925,t995,rh,u500,t850,season_fall,season_spring,season_summer,season_winter,t_diff
datetime,,,,,,,,,,,,,,,,,,,,,,,,
2016-06-01 02:00:00,67.00,16.65,-6.07,4.55,100807.00,-11.25,-7.50,-9.90,4.54,1.25,5.75,-0.84,-44.65,1.06,-4.46,20.02,42.30,34.50,-3.55,0.0,0.0,1.0,0.0,-3.37
2016-06-01 03:00:00,66.67,16.15,-6.43,4.48,100816.67,-11.15,-7.75,-13.23,4.08,1.24,5.33,-0.65,-44.68,0.42,-4.38,20.02,42.73,35.60,-3.45,0.0,0.0,1.0,0.0,-3.87
2016-06-01 04:00:00,66.33,15.65,-6.79,4.42,100826.33,-11.05,-7.99,-16.57,3.63,1.23,4.91,-0.45,-44.72,-0.21,-4.31,20.02,43.17,36.70,-3.35,0.0,0.0,1.0,0.0,-4.37
2016-06-01 05:00:00,66.00,15.15,-7.15,4.35,100836.00,-10.95,-8.24,-19.90,3.17,1.22,4.49,-0.26,-44.75,-0.85,-4.23,20.02,43.60,37.80,-3.25,0.0,0.0,1.0,0.0,-4.87
2016-06-01 06:00:00,71.67,18.08,-7.05,4.32,100858.00,-10.78,-8.74,-21.34,1.85,0.92,3.60,-0.25,-44.48,-1.05,-4.53,20.82,39.13,40.27,-3.02,0.0,0.0,1.0,0.0,-2.73


In [217]:
rnn = nn.LSTM(24,10,1)
lnn = nn.Linear(10,1)

output = torch.tensor([samples.iloc[0,0]])
for i in range(72):
    x_MeteFactor = torch.tensor(samples.iloc[i,1:].values)
    x_MeteFactor = x_MeteFactor.float()
    x_Onetime = torch.cat((output[-1].unsqueeze(0),x_MeteFactor)).unsqueeze(0).unsqueeze(0)
    o, (_1, _2) = rnn(x_Onetime)
    o = lnn(o)
    output = torch.cat((output,o[0,0].data),0)

In [218]:
output

tensor([67.0000, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473,
        -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473,
        -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473,
        -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473,
        -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473,
        -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473,
        -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473,
        -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473,
        -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473, -0.3473,
        -0.3473])

In [213]:
rnn = nn.LSTM(24,10,1)
lnn = nn.Linear(10,1)

output = torch.tensor([samples.iloc[0,0]])
for i in range(72):
    x_MeteFactor = torch.tensor(samples.iloc[i,1:].values)
    x_MeteFactor = x_MeteFactor.float()
    x_Onetime = torch.cat((output[-1].unsqueeze(0),x_MeteFactor)).unsqueeze(0).unsqueeze(0)
    o, (_1, _2) = rnn(x_Onetime)
    o = lnn(o)
    output = torch.cat((output,o[0,0].data),0)